Weighting sentences by their position in the review:

We calculate feature importance off of the test set's features because only the test set's sentences get scored (probssentences). Is this legal?

In [1]:
import pickle
import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [64]:
probssentences = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/probssentences.p', 'rb'))
test1 = pickle.load(open('/home/sarahwie/Documents/pubmed-nlp-research/pickled_objects/test1.p', 'rb'))

In [72]:
#append doc value to test1
test1['doc'] = [ i for i in range(len(test1))]
print test1.shape
print probssentences.shape

(12500, 4)
(133515, 3)


In [55]:
test1[0:10]

,id,sentiment,review,doc
21130,"""5706_2""",0.0,"""This movie is like the thousand \""cat and mou...",0
3404,"""11132_3""",0.0,"""A documentarist, like any filmmaker, must con...",1
14560,"""10770_7""",1.0,"""With a well thought out cast, this movie was ...",2
6671,"""1058_10""",1.0,"""this is the best sci-fi that I have seen in m...",3
10982,"""12239_1""",0.0,"""Ever since I was eight years old I have been ...",4
5526,"""966_7""",1.0,"""2005 Toronto Film Festival Report It is offic...",5
19301,"""11645_2""",0.0,"""This is a review of 'Freddy Mercury The Untol...",6
17060,"""5920_9""",1.0,"""American icon Henry Fonda portrays \""Elegant\...",7
24744,"""9501_2""",0.0,"""This is the page for \""House of Exorcism\"", b...",8
5253,"""7877_4""",0.0,"""The makers of this film have created a future...",9


In [52]:
#append the true class value
probssentences["truelabel"] = 0
for i in range(len(probssentences)):
    #get true label value from test1 where index matches doc number
    j = probssentences.iloc[i]['doc']
    probssentences.set_value(i, 'truelabel', test1.iloc[[j]]["sentiment"])

In [57]:
probssentences[0:10]

,0,1,doc,truelabel
0,0.5986,0.4014,0,0
1,0.00792023,0.99208,0,0
2,0.369248,0.630752,0,0
3,0.445318,0.554682,0,0
4,0.999581,0.000419095,0,0
5,0.00449226,0.995508,1,0
6,0.918007,0.0819928,1,0
7,0.55223,0.44777,1,0
8,0.986808,0.0131918,1,0
9,0.721142,0.278858,1,0


In [67]:
#split test set into two
fracTrain = 0.5
nSamples = test1.shape[0]
print nSamples
order = np.random.permutation(nSamples) # come up with a random ordering
print len(order)
splitIndex = int(np.round(nSamples*fracTrain))
print len(order[:splitIndex])
print len(order[splitIndex:])
testA = test1.iloc[order[:splitIndex],:]
testB = test1.iloc[order[splitIndex:],:]

(12500, 4)
12500
12500
6250
6250


In [69]:
#split probssentences the same way
probssentencesA = probssentences.loc[probssentences['doc'].isin(testA['doc'].unique())]
probssentencesB = probssentences.loc[probssentences['doc'].isin(testB['doc'].unique())]

In [71]:
print len(probssentences)
print len(probssentencesA)
print len(probssentencesB)
print len(testA)
print len(testB)

133515
66863
66652
6250
6250


In [75]:
probssentencesB[0:10]

,0,1,doc
47,0.615371,0.384629,6
48,0.0662093,0.933791,6
49,0.0751976,0.924802,6
50,0.995847,0.00415336,6
51,0.322619,0.677381,6
52,0.77551,0.224489,6
53,0.0407758,0.959224,6
54,0.93637,0.0636299,6
55,0.97243,0.0275697,6
56,0.00466961,0.99533,6


In [76]:
testA[0:10]

,id,sentiment,review,doc
8403,"""4316_1""",0,"""I'm not usually given to hyperbole, but after...",10239
9953,"""7233_7""",1,"""I haven't read through all the comments, but ...",2925
23611,"""9810_2""",0,"""It's hard to criticize this movie, because I ...",2849
1350,"""9683_9""",1,"""Spoiler This is a great film about a conure. ...",1295
13824,"""6672_10""",1,"""I've seen Mystery Men cop a bit of stick in t...",6490
1586,"""762_9""",1,"""I enjoy watching people doing breakdance, esp...",10367
1099,"""9626_8""",1,"""Warning! Spoilers ahead!<br /><br /> SPOILERS...",7118
12693,"""9538_10""",1,"""Why aren't more films (especially American) m...",7562
20256,"""3981_10""",1,"""This is my fourth Joe McDoakes short that I'v...",9709
20733,"""541_7""",1,"""as a sequel,this is not a bad movie.i actuall...",4421


In [77]:
testB[0:10]

,id,sentiment,review,doc
5332,"""7988_10""",1,"""I get teased all the time by family and frien...",1266
22736,"""3993_7""",1,"""\""The Mother\"" is a weird low-budget movie, t...",55
17040,"""893_2""",0,"""When I bought this DVD I though: \""It seems t...",9443
22406,"""1047_2""",0,"""Someone will have to explain to me why every ...",11748
8282,"""9342_8""",1,"""A powerfully wonderful movie. You are held in...",7742
17688,"""3532_1""",0,"""Its like if you took the general themes of Th...",7496
3522,"""12387_4""",0,"""Lipstick is another glossy movie failure.I am...",7408
11608,"""2793_9""",1,"""It's such a shame that because of it's title ...",10308
3335,"""2566_8""",1,"""Any film that deals with bigotry in a positiv...",11807
1235,"""11928_8""",1,"""This movie, while seemingly based off of a mo...",8666


### NOTE: probssentences and test sets are not in same order. Must match on doc number.

In [81]:
print len(testA['doc'].unique())

6250


In [118]:
features = []
labels = []
#for each doc in probssentences:
for doc in testA['doc'].unique():
    avgScores = []
    df = probssentencesA.loc[probssentences['doc'] == doc]
    #split into x groups
    lst = np.array_split(df[[0]], 3)
    for z in lst:
        print z
        print z.mean()
        #make the average score for each group the feature
        avgScores.append(z.mean()[0])
        #convert to a 0 or 1 (?)
    print avgScores
    features.append(avgScores)
    labels.append(testA.loc[testA['doc'] == doc]['sentiment'].values[0])
    
#perform feature weighting- START HERE MONDAY
clf = ExtraTreesClassifier()
clf = clf.fit(features, labels)
weights = clf.feature_importances_  
print weights

0
               0
109421  0.979633
109422  0.562139
109423  0.266345
109424  0.761915
109425  0.981866
109426  0.961239
0    0.75219
dtype: float64
               0
109427  0.993496
109428  0.971686
109429  0.787073
109430  0.766586
109431  0.998035
109432  0.970219
0    0.914516
dtype: float64
                0
109433  0.0874177
109434   0.621483
109435   0.730015
109436   0.942324
109437   0.984106
0    0.673069
dtype: float64
[0.75218957165877021, 0.9145159820715586, 0.67306907027959828]



In [ ]:
#put into function and make split number the parameter

In [ ]:
#apply on test set 2

In [39]:
preds = np.ones((prob.shape[0]))
print preds.shape

(12500,)


In [40]:
preds[np.where(prob.iloc[:,0] >  prob.iloc[:,1])] = 0 # The first column is the negative model

In [41]:
print preds.shape
print test1["sentiment"].shape

(12500,)
(12500,)


In [42]:
print np.size(np.where(preds == test1["sentiment"]))*1./np.size(preds)

0.86848


In [15]:
from sklearn.metrics import roc_auc_score
print roc_auc_score(test1["sentiment"], preds)

0.868160030313


Especially considering the weights are fit from the test data itself, we would expect to see a big increase in performance, which we're not getting.

Why? :/ could have something to do with the fact that we fit a classifier on the test data which is pretty wacky. Or could be the way the weights were applied to the sentence scores (unlikely). 